1. Si vuole predire il valore di range di prezzo sulla base degli attributi presenti nel dataset.
Ricaricare il dataset originale, eliminare eventuali attributi inutili (giustificare la scelta),
eliminare le eventuali istanze che contengono valori nulli, e dividere il dataset in modo che 3/4
degli elementi siano contenuti in un nuovo dataset “train” e 1/4 nel dataset “test” preservando
le proporzioni delle classi nella colonna target.
Allenare il train con il modello DecisionT ree e valutare l’accuracy ottenuta calcolata sia sul
dataset train sia sul dataset test. Confrontare i risultati ottenuti con quelli ottenuti con una
predizione basata sul modello KNeighborsClassifier . Effettuare alcune considerazioni sui risultati
ottenuti, tenendo in considerazione anche l’analisi della confusion matrix e la predizione
effettuata da un dummy classifier a scelta. (punti 4)

In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.dummy import DummyClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import KBinsDiscretizer, StandardScaler

from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier


In [122]:
df = pd.read_csv('dataset.csv')
df

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


In [123]:
X = df.drop('price_range', axis=1)
y = df['price_range']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=df['price_range'], test_size=0.25, random_state=42)

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_train)
train_dt_accuracy = accuracy_score(y_train, y_pred)
print(f'TRAIN tree: {train_dt_accuracy } ')
y_pred = dt.predict(X_test)
test_dt_accuracy = accuracy_score(y_test, y_pred)
print(f'TEST tree: {test_dt_accuracy}')
print(confusion_matrix(y_test, y_pred))

kn = KNeighborsClassifier()
kn.fit(X_train, y_train)
y_pred = kn.predict(X_train)

train_knn_accuracy = accuracy_score(y_train, y_pred)
print(f'TRAIN KNN: {train_knn_accuracy } ')
y_pred = kn.predict(X_test)
test_knn_accuracy = accuracy_score(y_test, y_pred)
print(f'TEST KNN: {test_knn_accuracy}')
print(confusion_matrix(y_test, y_pred))

dummy = DummyClassifier()
dummy.fit(X_train, y_train)
y_pred = dummy.predict(X_train)
print(f'TRAIN Dummy: {accuracy_score(y_train,y_pred) }')
y_pred = dummy.predict(X_test)
print(f'TEST Dummy: {accuracy_score(y_test,y_pred)}')
print(confusion_matrix(y_test, y_pred))

TRAIN tree: 1.0 
TEST tree: 0.824
[[113  12   0   0]
 [ 18  93  14   0]
 [  0  11  99  15]
 [  0   0  18 107]]
TRAIN KNN: 0.94 
TEST KNN: 0.942
[[123   2   0   0]
 [  2 118   5   0]
 [  0   6 113   6]
 [  0   0   8 117]]
TRAIN Dummy: 0.25
TEST Dummy: 0.25
[[125   0   0   0]
 [125   0   0   0]
 [125   0   0   0]
 [125   0   0   0]]


Dato che il decision tree è perfetto sul train, ma sbaglia sul test c'è overfitting,
il knn, invece ha prestazioni peggiori sul train, ma migliori sul test, quindi non c'è overfitting per questo dataset.
Il dummy fa cacare

# 2

2. Confrontare l’accuratezza ottenuta nel punto precedente con l’accuratezza che si ottiene con
una 10 Fold cross validation. (punti 1)

In [124]:
results = cross_val_score(dt, X, y, cv=10, scoring='accuracy', n_jobs=-1)
print(f'CV Accuracy: {results.mean()*100}%')

CV Accuracy: 83.6%


# 3
3. T rovare i parametri migliori del classificatore DecisionT ree. Agire sui parametri criterion e
min
_
samples
_
leaf. Verificare se l’accuratezza che si ottiene con la nuova configurazione supera
quella ottenuta con i parametri di default al punto 1. (punti 3)

In [125]:
params = {
    'criterion' : ['gini', 'entropy'],
    'min_samples_leaf' : [2, 4, 6],
}

gs = GridSearchCV(dt, params, cv=10, scoring='accuracy')
gs.fit(X_train, y_train)
y_pred = gs.predict(X_test)

print(f"Migliori parametri trovati: {gs.best_params_}")
grid_dt_accuracy = accuracy_score(y_test, y_pred)
print(f"Grid search Accuracy (Test Set): {grid_dt_accuracy} ")
print(f'Plain Decision Tree Accuracy (Test Set): {test_dt_accuracy} ')

Migliori parametri trovati: {'criterion': 'entropy', 'min_samples_leaf': 4}
Grid search Accuracy (Test Set): 0.892 
Plain Decision Tree Accuracy (Test Set): 0.824 


# 4
Studiare la correlazione tra tutte le feature del dataset, creare un dataframe che contiene,
oltre alla colonna target, le 5 feature più correlate (positivamente) al target. Ripetere la
classificazione sul nuovo dataset e verificare se l’accuratezza ottenuta con DecisionT ree e
KNeighborsClassifier migliora (punti 3).

In [126]:
correlation_matrix = df.corr()
print(correlation_matrix['price_range'].sort_values(ascending=False))
print(correlation_matrix['price_range'].sort_values(ascending=False)[0:6].index.tolist())
top_indexes = correlation_matrix['price_range'].sort_values(ascending=False)[0:6].index.tolist()
df_top5 = df[top_indexes]
df_top5

price_range      1.000000
ram              0.917046
battery_power    0.200723
px_width         0.165818
px_height        0.148858
int_memory       0.044435
sc_w             0.038711
pc               0.033599
three_g          0.023611
sc_h             0.022986
fc               0.021998
talk_time        0.021859
blue             0.020573
wifi             0.018785
dual_sim         0.017444
four_g           0.014772
n_cores          0.004399
m_dep            0.000853
clock_speed     -0.006606
mobile_wt       -0.030302
touch_screen    -0.030411
Name: price_range, dtype: float64
['price_range', 'ram', 'battery_power', 'px_width', 'px_height', 'int_memory']


,price_range,ram,battery_power,px_width,px_height,int_memory
0,1,2549,842,756,20,7
1,2,2631,1021,1988,905,53
2,2,2603,563,1716,1263,41
3,2,2769,615,1786,1216,10
4,1,1411,1821,1212,1208,44
...,...,...,...,...,...,...
1995,0,668,794,1890,1222,2
1996,2,2032,1965,1965,915,39
1997,3,3057,1911,1632,868,36
1998,0,869,1512,670,336,46


In [127]:
X = df_top5.drop('price_range', axis=1)
y = df_top5['price_range']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
test_top5_dt_accuracy = accuracy_score(y_test, y_pred)
print(f'TEST top 5 DT: {test_top5_dt_accuracy}')
print(f'TEST plain DT: {test_dt_accuracy}')
# print(confusion_matrix(y_test, y_pred))

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
test_top5_knn_accuracy = accuracy_score(y_test, y_pred)
print(f'TEST top 5 KNN: {test_top5_knn_accuracy}')
print(f'TEST KNN: {test_knn_accuracy}')

TEST top 5 DT: 0.254
TEST plain DT: 0.824
TEST top 5 KNN: 0.932
TEST KNN: 0.942


Possiamo notare che il decision tree migliora, perché usando meno features si riduce l'overfitting.
Mentre il knn perde molte informazioni, ma non aveva overfitting, quindi peggiora le sue prestazioni

# 5
5. Creare una pipeline in cui gli attributi intmemory, ram e talktime sono scalati in
_
_
modo che abbiano media 0 e varianza 1, gli attributi mobile
wt e battery
power sono
_
_
discretizzati in 5 intervalli, e tutti gli altri attributi sono lasciati invariati. La pipeline deve
applicare il modello DecisionT ree con i parametri migliori trovati al punto 2. Valutare
l’accuratezza della classificazione. (punti 3)

In [128]:
X = df.drop('price_range', axis=1)
y = df['price_range']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)


preprocessing = ColumnTransformer(
    transformers = [
    ('scaler', StandardScaler(), ['int_memory', 'ram', 'talk_time']),
    ('discretizer', KBinsDiscretizer(n_bins=5), ['mobile_wt', 'battery_power'])
], remainder='passthrough'
)

model = DecisionTreeClassifier(**gs.best_params_)

pipeline = Pipeline(
    steps=[
        ('preprocessing', preprocessing),
        ('model', model)
    ]
)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
test_pipeline_acc = accuracy_score(y_test, y_pred)
print(f'TEST Pipeline con DT: {test_pipeline_acc}')
print(f'TEST top 5 DT: {test_top5_dt_accuracy}')


TEST Pipeline con DT: 0.852
TEST top 5 DT: 0.254


/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


Ovviamente i parametri erano i migliori per il df senza scaler e disc, quindi ora non sono più i migliori, quindi fa cacare.
Inoltre ora ho di nuovo l'overfitting.
La discretizzazione non fa bene al decision tree in quanto comporta una perdita di informazione, soprattutto su battery power, che è una top5.
Lo scaler invece non ha minimamente effetto sul decision tree

# 6
Aggiungere alla pipeline del punto 5 la funzione SelectKBest
(https://scikit-learn.org/stable/modules/generated/sklearn.feature
selection.SelectKBest.html
_
?highlight=selectkbest#sklearn.feature
_
selection.SelectKBest). Utilizzare la funzione di
gridSearchCV per selezionare il K migliore e anche il numero migliore di bin in cui discretizzare i
valori di mobile
wt e battery
power (scegliere a piacere alcuni valori) (punti 3).

In [129]:
pipeline_v2 = Pipeline(
    steps=[
        ('preprocessing', preprocessing),
        ('kbest', SelectKBest()),
        ('model', model)
    ]
)

param_grid = {
    'kbest__k': [2, 3, 4],
    'preprocessing__discretizer__n_bins': [3, 5, 10],
}

gs = GridSearchCV(pipeline_v2, param_grid, cv=10, scoring='accuracy', n_jobs=-1)
gs.fit(X_train, y_train)
y_pred = gs.predict(X_test)
grid_kbest_acc_train = gs.best_score_
grid_kbest_acc = accuracy_score(y_test, y_pred)

grid_kbest_params = gs.best_params_
print(f'Grid Search Accuracy test: {grid_kbest_acc}, Grid Search Params: {grid_kbest_params}, Grid Search Kbest: {grid_kbest_acc_train}')

/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/preprocessing/_discretization.py:296: 

Grid Search Accuracy test: 0.868, Grid Search Params: {'kbest__k': 4, 'preprocessing__discretizer__n_bins': 3}, Grid Search Kbest: 0.8206666666666667


/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


# 7
Creare una nuova pipeline che applica la decomposizione T runcatedSVD (simile alla PCA -
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.T runcatedSVD.html)
al dataset iniziale e aggiunge le componenti ottenute alla pipeline del punto 5. Valutare il valore
migliore per il numero di componenti di T runcatedSVD tra 2, 4 e 6. (punti 3).

In [130]:
best_n_bins = grid_kbest_params['preprocessing__discretizer__n_bins']

preprocessing = ColumnTransformer(
    transformers = [
    ('scaler', StandardScaler(), ['int_memory', 'ram', 'talk_time']),
    ('discretizer', KBinsDiscretizer(n_bins=best_n_bins), ['mobile_wt', 'battery_power'])
], remainder='passthrough'
)

combined_features = FeatureUnion([("truncatedSVD", TruncatedSVD()), ("preprocessing", preprocessing)])

pipeline_v3 = Pipeline(
    steps=[
        ('combined_features', combined_features),
        ('model', model)
    ]
)

param_grid = {
    'combined_features__truncatedSVD__n_components': [2, 4, 6],
}
gs = GridSearchCV(pipeline_v3, param_grid, cv=10, scoring='accuracy')
gs.fit(X_train, y_train)
y_pred = gs.predict(X_test)
grid_kbest_acc_train = gs.best_score_
grid_kbest_acc = accuracy_score(y_test, y_pred)

grid_kbest_params = gs.best_params_

print(f'Grid Search Accuracy test: {grid_kbest_acc}, Grid Search Params: {grid_kbest_params}, Grid Search Kbest: {grid_kbest_acc_train}')


/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/preprocessing/_discretization.py:296: 

Grid Search Accuracy test: 0.956, Grid Search Params: {'combined_features__truncatedSVD__n_components': 4}, Grid Search Kbest: 0.9386666666666666


/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/preprocessing/_discretization.py:296: 